In [41]:
import os
import glob

folder_path = '1kfiles'
data = []

for filename in glob.glob(os.path.join(folder_path, '*.csv')):
    try:
        df = pd.read_csv( filename, header = None,
                         names=['des_flux', 'detrap_en', 'def_conc'],
                         index_col =None, squeeze = True) 
        df['filename'] = filename
        data.append(df)
    except pd.errors.ParserError as e:
        print(f"Error reading file {filename}: {e}")

df_all=pd.concat(data, axis = 0 , ignore_index= True)
df_all = df_all.dropna()
display(df_all)

C:\Users\yayy_\AppData\Local\Temp\ipykernel_28152\3686693509.py:9: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  df = pd.read_csv( filename, header = None,
C:\Users\yayy_\AppData\Local\Temp\ipykernel_28152\3686693509.py:9: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  df = pd.read_csv( filename, header = None,
C:\Users\yayy_\AppData\Local\Temp\ipykernel_28152\3686693509.py:9: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  df = pd.read_csv( filename, header = None,
C:\Users\yayy_\AppData\Local\Temp\ipykernel_28152\3686693509.py:9: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.

,des_flux,detrap_en,def_conc,filename
0,1.269745e-10,1.051587,0.011064,1kfiles\training_data_1.csv
1,8.488521e+14,1.185898,0.006066,1kfiles\training_data_1.csv
2,1.239766e+15,1.203310,0.020460,1kfiles\training_data_1.csv
3,1.784721e+15,1.242848,0.005844,1kfiles\training_data_1.csv
301,0.000000e+00,1.255962,0.011071,1kfiles\training_data_10.csv
...,...,...,...,...
300401,4.169630e+15,1.997682,0.018687,1kfiles\training_data_98.csv
300699,0.000000e+00,1.000740,0.005143,1kfiles\training_data_99.csv
300700,3.823177e+15,1.275135,0.001468,1kfiles\training_data_99.csv
300701,5.083735e+15,1.357167,0.006096,1kfiles\training_data_99.csv


In [42]:
#training data
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

#Split into train and test sets
train_df , test_df = train_test_split(df_all , test_size = 0.1, random_state = 42)

# Drop filename column from train_df
train_df = train_df.drop(columns = ['filename'])
test_df = test_df.drop(columns = ['filename'])

#Define the data preprocessing steps and the model
preprocessor = make_pipeline(StandardScaler())
model = LinearRegression()

#Split the data into input features and target variable
X_train = train_df.drop(columns = ['detrap_en', 'def_conc'])
y_train = train_df[['detrap_en','def_conc']]

X_test = test_df.drop(columns = ['detrap_en', 'def_conc'])
y_test = test_df[['detrap_en', 'def_conc']]

# Fit the model using cross-validation
pipeline = make_pipeline(preprocessor, model)
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(f"Cross-validation score: {cv_scores}")
print(f"Mean cross-validation score: {cv_scores.mean()}")

# Fit the model on the entire training set and evaluate on the test set
pipeline.fit(X_train, y_train)
test_score = pipeline.score(X_test, y_test)
print(f"Test set score : {test_score}")

Cross-validation score: [-0.04377912 -0.04204896 -0.04201728 -0.04264947 -0.04129857]
Mean cross-validation score: -0.04235867958247323
Test set score : -0.0015404997230011874


In [44]:
# Scale the data using StandardScaler
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_df.drop(columns=['detrap_en', 'def_conc']))
test_scaled = scaler.transform(test_df.drop(columns=['detrap_en', 'def_conc']))

# Define the MLP model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=128, activation='relu', input_shape=(train_scaled.shape[1],)),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=2, activation='linear')
])

# Compile the model
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0001), loss='mse')

# Train the model
history = model.fit(train_scaled, train_df[['detrap_en', 'def_conc']].values, epochs=100, batch_size=64, validation_split=0.1)

# Evaluate the model on the test set
test_loss = model.evaluate(test_scaled, test_df[['detrap_en', 'def_conc']].values)
print(f'Test loss: {test_loss:.4f}')

Epoch 1/100
51/51 [==============================] - 2s 5ms/step - loss: 1.0390 - val_loss: 1.0076
Epoch 2/100
51/51 [==============================] - 0s 2ms/step - loss: 0.8600 - val_loss: 0.8058
Epoch 3/100
51/51 [==============================] - 0s 2ms/step - loss: 0.6441 - val_loss: 0.5645
Epoch 4/100
51/51 [==============================] - 0s 2ms/step - loss: 0.4039 - val_loss: 0.3013
Epoch 5/100
51/51 [==============================] - 0s 2ms/step - loss: 0.1917 - val_loss: 0.1262
Epoch 6/100
51/51 [==============================] - 0s 2ms/step - loss: 0.0965 - val_loss: 0.0780
Epoch 7/100
51/51 [==============================] - 0s 2ms/step - loss: 0.0796 - val_loss: 0.0698
Epoch 8/100
51/51 [==============================] - 0s 2ms/step - loss: 0.0744 - val_loss: 0.0654
Epoch 9/100
51/51 [==============================] - 0s 2ms/step - loss: 0.0696 - val_loss: 0.0607
Epoch 10/100
51/51 [==============================] - 0s 2ms/step - loss: 0.0653 - val_loss: 0.0568
Epoch 11/